In [0]:
%run  "./stocks_functions_helper"

In [0]:
dbutils.widgets.dropdown('month', '1', [str(x) for x in range(1,13)], 'Select Month')
dbutils.widgets.dropdown('year','2021',
       choices=['2018','2019','2020','2021','2022'],label='Select Year')
dbutils.widgets.text('day','Enter appropriate day','Day')

yep

In [0]:
v_selected_month = dbutils.widgets.get('month')
v_selected_year = dbutils.widgets.get('year')
v_selected_day = dbutils.widgets.get('day')
print(f'selected year: {v_selected_year}, month: {v_selected_month}, day: {v_selected_day}')

selected year: 2020, month: 1, day: 2

In [0]:
print("checking to see it is a valid data that's been punched in")
if validate_date(v_selected_year, v_selected_month, v_selected_day):
    print('valid data')
else:
    print('check the date used')
    dbutils.notebook.exit('invalid date')

checking to see it is a valid data that's been punched in
valid data

In [0]:
fo_data_location = f'/mnt/fopart/year={v_selected_year}/month={v_selected_month}/day={v_selected_day}'
print(fo_data_location)

fo_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv(fo_data_location)\
    .withColumn('year', lit(v_selected_year).cast('int'))\
    .withColumn('month', lit(v_selected_month).cast('int'))\
    .withColumn('day', lit(v_selected_day).cast('int'))

/mnt/fopart/year=2020/month=1/day=2

In [0]:
fodf_grouped_instrument = group_df(fo_df)

In [0]:
fodf_pcr = add_pcr_to_df(fodf_grouped_instrument)

In [0]:
fodf_processed = combine_grouped_and_pcr_dfs(
    fodf_grouped_instrument,fodf_pcr)

In [0]:
spark.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [0]:
print('number of rows in fo processed table for the selected dates before executing')
spark.sql(f'''
select count(*) as dayreccount from fopr where year = {v_selected_year} 
and month = {v_selected_month} 
and day = {v_selected_day}
''').show()

number of rows in fo processed table for the selected dates before executing
+-----------+
dayreccount|
+-----------+
 1363|
+-----------+

In [0]:
fodf_processed.write.mode('overwrite').insertInto('fopr')

In [0]:
print('number of rows in fo processed table after the partition created for the dates in question')
spark.sql(f'''
select count(*) as dayreccount from fopr where year = {v_selected_year} 
and month = {v_selected_month} 
and day = {v_selected_day}
''').show()

number of rows in fo processed table after the partition created for the dates in question
+-----------+
dayreccount|
+-----------+
 1363|
+-----------+

In [0]:
spark.sql(f'''
select * from fopr where year = {v_selected_year} 
and month = {v_selected_month} 
and day = {v_selected_day}
and symbol = 'NIFTY'
''').show(5, False)

+-------------------+------+-----------+----------+----------+---------+-----------------+-----------+------------------+----+-----+---+
rts |SYMBOL|EXPIRY_DT |INSTRUMENT|OPTION_TYP|CONTRACTS|VALUE |OPEN_INT |PCR |YEAR|MONTH|DAY|
+-------------------+------+-----------+----------+----------+---------+-----------------+-----------+------------------+----+-----+---+
2020-01-08 00:00:00|NIFTY |30-Jan-2020|FUTIDX |XX |153324.0 |1384001.33 |1.20027E7 |1.1691554135365299|2020|1 |8 |
2020-01-08 00:00:00|NIFTY |30-Jan-2020|OPTIDX |CE |390558.0 |3621734.099999999|1.8083025E7|1.1691554135365299|2020|1 |8 |
2020-01-08 00:00:00|NIFTY |30-Jan-2020|OPTIDX |PE |456623.0 |4080872.619999999|2.30742E7 |1.1691554135365299|2020|1 |8 |
2020-01-08 00:00:00|NIFTY |05-Mar-2020|OPTIDX |CE |0.0 |0.0 |0.0 |null |2020|1 |8 |
2020-01-08 00:00:00|NIFTY |29-Dec-2022|OPTIDX |CE |0.0 |0.0 |112500.0 |null |2020|1 |8 |
+-------------------+------+-----------+----------+----------+---------+-----------------+-----------+------------------+----+-----+---+
only showing top 5 rows

In [0]:
dbutils.notebook.exit('success')

success